# Export Risk and Ambiguity Data

This script outputs risk and ambiguity data from MAT files created by the aging scripts. The data is extracted, processed, and saved into CSV files for further analysis. The process is divided into two main sections: handling the monetary task data and handling the medical task data.

---

## Load Required Libraries

First, we load the necessary libraries for data manipulation and file handling.

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from glob import glob

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## Define File Path and Initialize DataFrames

We define the file path pattern for the monetary task MAT files and initialize empty DataFrames to store the extracted data and any missing data.

In [2]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Risk_and_Ambiguity_new/AG_*_RA/MDM_MON_*.mat'
df = pd.DataFrame()
df_missing = pd.DataFrame()

## Extract and Process Monetary Task Data

We iterate over each MAT file matching the specified pattern, load the data, and extract the relevant information: values, risk, ambiguity, and choice. The extracted data is then organized into DataFrames.

In [3]:
for mat_fname in glob(glober):
    # Load the MAT file
    mat = sio.loadmat(mat_fname)
    # Extract the subject number from the file name
    sub = mat_fname.split('_')[5]

    # Extract values, risk, and ambiguity data from the MAT file
    vals = pd.DataFrame(mat['Datamon'][0][0][8], columns=['value'])
    risk = pd.DataFrame(mat['Datamon'][0][0][9], columns=['risk'])
    ambiguity = pd.DataFrame(mat['Datamon'][0][0][10], columns=['ambiguity'])

    # Initialize lists to store choice and value data for each block
    choice = []
    value = []
    # Iterate over each block and each trial within the block
    for j in range(4):
        for i in range(21):
            # Append the choice data
            choice.append(mat['Datamon'][0][0][17][j][i][0][0])
            # Append the value data, ensuring correct values are used
            if mat['Datamon'][0][0][17][j][i][2][0] == '$0':
                value.append(mat['Datamon'][0][0][17][j][i][3][0])
            else:
                value.append(mat['Datamon'][0][0][17][j][i][2][0])

    # Create a DataFrame for the current subject's data
    db = pd.DataFrame(list(zip(choice, value)), columns=['choice', 'value'])

    # Add values, risk, ambiguity, and subject number to the DataFrame
    db['value'] = vals
    db['risk'] = risk
    db['ambiguity'] = ambiguity
    db['sub'] = sub

    # Replace textual choices with numerical codes
    db['choice'] = db.choice.replace(['Reference'], 0)
    db['choice'] = db.choice.replace(['Lottery'], 1)
    db['choice'] = db.choice.replace(['None'], np.nan)

    # Add reaction time data to the DataFrame
    db['rt'] = pd.DataFrame(mat['Datamon']['rt'][0][0][0], columns=['RT'])

    # Check for catch trials (where the choice is between $5 for sure and a chance for $5) 
    # Print the subject number if they chose the lottery on more than 5 (out of 10) occasions. 
    catch = np.nansum(db.choice[db.value == 5])
    if catch > 5:
        print(sub, catch)
    db['catch'] = catch

    # Concatenate the current subject's data to the main DataFrame
    df = pd.concat([df, db])

12 10
17 11
27 11
29 6
99 6.0
111 8


Six participants failed the catch trials.

## Save Extracted Monetary Data to CSV

We save the concatenated data from all subjects into a CSV file.

In [4]:
df.to_csv('data/mon.csv')

## Define File Path and Initialize DataFrames for Medical Task

Similarly, we define the file path pattern for the medical task MAT files and re-initialize empty DataFrames for storing the data.

In [5]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Risk_and_Ambiguity_new/AG_*_RA/MDM_MED_*.mat'
df = pd.DataFrame()
df_missing = pd.DataFrame()

## Extract and Process Medical Task Data

We iterate over each MAT file matching the pattern for the medical task, load the data, and extract relevant information such as values, risk, and ambiguity. The extracted data is organized into DataFrames.

In [6]:
for mat_fname in glob(glober):

    # Load the MAT file
    mat = sio.loadmat(mat_fname)
    # Extract the subject number from the file name
    sub = mat_fname.split('_')[5]

    # Extract values, risk, and ambiguity data from the MAT file
    vals = pd.DataFrame(mat['Datamed'][0][0][8], columns = ['value'])
    risk = pd.DataFrame(mat['Datamed'][0][0][9], columns = ['risk']) 
    ambiguity = pd.DataFrame(mat['Datamed'][0][0][10], columns = ['ambiguity'])
    
    # Initialize lists to store choice and value data for each block
    choice = []
    value = []
    # Iterate over each block and each trial within the block
    try:
        for j in range(4):
            for i in range(21):    
                choice.append(mat['Datamed'][0][0][17][j][i][0][0])

                if mat['Datamed'][0][0][17][j][i][2][0] == 'no effect':
                    value.append(mat['Datamed'][0][0][17][j][i][3][0])
                else:
                    value.append(mat['Datamed'][0][0][17][j][i][2][0])
    except:
        print("failed choice", sub)
        continue
        
    # Create a DataFrame for the current subject's data
    db = pd.DataFrame(list(zip(choice, value)),
               columns =['choice', 'value'])
    
    # Add values, risk, ambiguity, and subject number to the DataFrame
    db['value'] = vals
    db['risk'] = risk
    db['ambiguity'] = ambiguity
    db['sub'] = sub

     # Replace textual choices with numerical codes
    db['choice'] = db.choice.replace(['Conservative treatment: Slight improvement'], 0)
    db['choice'] = db.choice.replace(['Experimental treatment'], 1)
    db['choice'] = db.choice.replace(['None'], np.nan)

    # Add reaction time data to the DataFrame
    db['rt'] = pd.DataFrame(mat['Datamed']['rt'][0][0][0], columns = ['RT'])
    
    # Check for catch trials (where the choice is between a slight improvement for sure and a chance for a slight improvement) 
    # Print the subject number if they chose the lottery on more than 5 (out of 10) occasions. 
    catch = np.nansum(db.choice[db.value == 5])
    if catch > 5:
        print(sub, catch)
    db['catch'] = catch
    
    # Concatenate the current subject's data to the main DataFrame
    df = pd.concat([df,db])

22 8.0
12 12
17 6
failed choice 28
60 6
88 6.0
112 7
111 9


Seven subjects failed the catch trials. One participant file corrupted.

## Save Extracted Medical Data to CSV

We save the concatenated data from all subjects into a CSV file.

In [7]:
df.to_csv('data/med.csv')

## Watermark

Finally, we load the watermark extension to keep track of the notebook metadata.

In [8]:
%load_ext watermark
%watermark -n -u -v -iv -w -p xarray

Last updated: Mon Aug 05 2024

Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.26.0

xarray: 2024.7.0

pandas: 2.2.2
scipy : 1.12.0
numpy : 1.23.5

Watermark: 2.4.3

